In [1]:
import pandas as pd

In [2]:
projects_time_series = pd.read_pickle('../../data/02_prepared_data/projects_time_series.pkl')
projects_time_series.head()

,project_id,name,language,created_at,year,month,months_from_create
13,1,akka,Scala,2009-02-16 12:51:54,2009,2,0
14,1,akka,Scala,2009-02-16 12:51:54,2009,3,1
15,1,akka,Scala,2009-02-16 12:51:54,2009,4,2
16,1,akka,Scala,2009-02-16 12:51:54,2009,5,3
17,1,akka,Scala,2009-02-16 12:51:54,2009,6,4


Usunięcie zbędnych kolumn i zresetowanie indeksów

**TODO: CO Z JĘZYKIEM PROGRAMOWANIA?**

In [3]:
projects_time_series = projects_time_series.reset_index().drop(columns={'index', 'name', 'created_at'})
projects_time_series.head()

,project_id,language,year,month,months_from_create
0,1,Scala,2009,2,0
1,1,Scala,2009,3,1
2,1,Scala,2009,4,2
3,1,Scala,2009,5,3
4,1,Scala,2009,6,4


In [4]:
def merge_into_projects_time_series(df1, df2):
    return pd.merge(df1, df2, on=['project_id', 'year', 'month'], how='left', sort=False)

In [5]:
def fill_NaN_for_monthly_values(df, col_name):
    return df[col_name].fillna(0)

In [6]:
def fill_NaN_for_summary_values(df, col_name):
    return df.groupby('project_id')[col_name].ffill().fillna(0)

In [7]:
def full_merge(df1, df2, month_val, summ_val):
    new_df = merge_into_projects_time_series(df1, df2)
    new_df[month_val] = fill_NaN_for_monthly_values(new_df, month_val)
    new_df[month_val] = new_df[month_val].astype('int64')
    new_df[summ_val] = fill_NaN_for_summary_values(new_df, summ_val)
    new_df[summ_val] = new_df[summ_val].astype('int64')
    return new_df

In [8]:
def merge_monthly_value(df1, df2, month_val):
    new_df = merge_into_projects_time_series(df1, df2)
    new_df[month_val] = fill_NaN_for_monthly_values(new_df, month_val)
    new_df[month_val] = new_df[month_val].astype('int64')
    return new_df

In [9]:
def merge_summary_value(df1, df2, summ_val):
    new_df = merge_into_projects_time_series(df1, df2)
    new_df[summ_val] = fill_NaN_for_summary_values(new_df, summ_val)
    new_df[summ_val] = new_df[summ_val].astype('int64')
    return new_df

In [10]:
# commity
new_commits = pd.read_pickle('../../data/02_prepared_data/new_commits.pkl')
new_issues = pd.read_pickle('../../data/02_prepared_data/new_issues.pkl')
# issues
unique_committers = pd.read_pickle('../../data/02_prepared_data/unique_committers.pkl')
new_issue_comments = pd.read_pickle('../../data/02_prepared_data/new_issue_comments.pkl')
# pull requesty zmergowane
new_opened_pull_requests_to_merge = pd.read_pickle('../../data/02_prepared_data/new_opened_pull_requests_to_merge.pkl')
new_merged_pull_requests = pd.read_pickle('../../data/02_prepared_data/new_merged_pull_requests.pkl')
new_closed_merged_pull_requests = pd.read_pickle('../../data/02_prepared_data/new_closed_merged_pull_requests.pkl')
total_merged_pull_requests = pd.read_pickle('../../data/02_prepared_data/total_merged_pull_requests.pkl')
# pull requesty niezmergowane
new_opened_pull_requests_to_discard = pd.read_pickle('../../data/02_prepared_data/new_opened_pull_requests_to_discard.pkl')
new_closed_unmerged_pull_requests = pd.read_pickle('../../data/02_prepared_data/new_closed_unmerged_pull_requests.pkl')
total_unmerged_pull_requests = pd.read_pickle('../../data/02_prepared_data/total_unmerged_pull_requests.pkl')

In [11]:
# commity
projects_time_series = full_merge(projects_time_series, new_commits, 'new_commits', 'total_commits')
projects_time_series = full_merge(projects_time_series, new_issues, 'new_issues', 'total_issues')
# issues
projects_time_series = full_merge(projects_time_series, unique_committers, 'unique_committers', 'total_unique_committers')
projects_time_series = full_merge(projects_time_series, new_issue_comments, 'new_issue_comments', 'total_issue_comments')
# pull requesty zmergowane
projects_time_series = merge_monthly_value(projects_time_series, new_opened_pull_requests_to_merge, 'new_opened_pull_requests_to_merge')
projects_time_series = merge_monthly_value(projects_time_series, new_merged_pull_requests, 'new_merged_pull_requests')
projects_time_series = merge_monthly_value(projects_time_series, new_closed_merged_pull_requests, 'new_closed_merged_pull_requests')
projects_time_series = merge_summary_value(projects_time_series, total_merged_pull_requests, 'total_merged_pull_requests')
# pull requesty niezmergowane
projects_time_series = merge_monthly_value(projects_time_series, new_opened_pull_requests_to_discard, 'new_opened_pull_requests_to_discard')
projects_time_series = merge_monthly_value(projects_time_series, new_closed_unmerged_pull_requests, 'new_closed_unmerged_pull_requests')
projects_time_series = merge_summary_value(projects_time_series, total_unmerged_pull_requests, 'total_unmerged_pull_requests')

projects_time_series.head()

,project_id,language,year,month,months_from_create,new_commits,total_commits,new_issues,total_issues,unique_committers,total_unique_committers,new_issue_comments,total_issue_comments,new_opened_pull_requests_to_merge,new_merged_pull_requests,new_closed_merged_pull_requests,total_merged_pull_requests,new_opened_pull_requests_to_discard,new_closed_unmerged_pull_requests,total_unmerged_pull_requests
0,1,Scala,2009,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,Scala,2009,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,Scala,2009,4,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,Scala,2009,5,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,Scala,2009,6,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
projects_time_series[projects_time_series['unique_committers'] > 20]

,project_id,language,year,month,months_from_create,new_commits,total_commits,new_issues,total_issues,unique_committers,total_unique_committers,new_issue_comments,total_issue_comments,new_opened_pull_requests_to_merge,new_merged_pull_requests,new_closed_merged_pull_requests,total_merged_pull_requests,new_opened_pull_requests_to_discard,new_closed_unmerged_pull_requests,total_unmerged_pull_requests
316,9,C++,2012,3,38,82,469,15,199,25,61,36,182,5,5,5,50,10,6,85
321,9,C++,2012,8,43,38,629,23,297,21,75,8,314,9,9,9,69,14,3,132
407,12,C++,2011,6,6,483,10506,16,60,26,54,33,97,6,6,6,15,10,8,41
408,12,C++,2011,7,7,301,10807,7,67,24,59,13,110,0,0,0,15,7,7,48
409,12,C++,2011,8,8,336,11143,60,127,41,78,98,208,32,28,28,43,28,23,71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1234330,78852,Ruby,2013,4,60,49,228,427,11137,21,51,1769,47979,175,182,182,4042,89,66,2183
1234332,78852,Ruby,2013,6,62,55,342,441,12037,25,69,1895,51780,156,162,162,4355,108,88,2332
1234333,78852,Ruby,2013,7,63,98,440,548,12585,30,81,2195,53975,197,204,204,4559,147,109,2441
1234334,78852,Ruby,2013,8,64,148,588,429,13014,53,115,1784,55759,135,145,145,4704,133,69,2510


## Przykład - połączenie danych commitów i wypełnienie braków danych

In [13]:
# new_commits = pd.read_pickle('../../data/02_prepared_data/new_commits.pkl')
# new_commits.head()

In [14]:
# time_series_with_commits_info = pd.merge(projects_time_series, new_commits, on=['project_id', 'year', 'month'], how='left', sort=False)
# time_series_with_commits_info.isnull().sum()

Jaki procent wierszy ma jakiekolwiek commity

In [15]:
# (1810601 - 1793145)/1810601 * 100

Obsługa brakujących danych:
- dla kolumny `new_commits` w miejscu braków uzupełniamy zerem
- dla kolumny `total_commits` w miejscu braków uzupełniamy wartością z wiersza wyżej

In [16]:
# time_series_with_commits_info['new_commits'].fillna(value=0, inplace=True)
# time_series_with_commits_info.isnull().sum()

In [17]:
# time_series_with_commits_info['new_commits'] = time_series_with_commits_info['new_commits'].astype('int64')
# time_series_with_commits_info

In [18]:
# pd.isna(time_series_with_commits_info.loc[0]['total_commits'])

In [19]:
# time_series_with_commits_info['total_commits'] = \
#     time_series_with_commits_info.groupby('project_id')['total_commits'].ffill().fillna(0)
# time_series_with_commits_info['total_commits'] = time_series_with_commits_info['total_commits'].astype('int64')
# time_series_with_commits_info.isnull().sum()

Sprawdzenie poprawności wypełnienia braków

In [20]:
# time_series_with_commits_info[
#     (time_series_with_commits_info['project_id'] == 5)]